In [1]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Московский рейтинг университетов

In [7]:
years = list(range(2020, 2022 + 1))
msk_data = []

for y in years:
    # Обратите внимание на вариативность в url (последний год отличается от всех остальных)
    msk_url = f"https://mosiur.org/{'ranking' if y == 2022 else 'ranking' + str(y)}/"
    msk_resp = requests.get(msk_url)

    if msk_resp.status_code == 200:
        msk_soup = BeautifulSoup(msk_resp.text)

        for tr in msk_soup.select('#top_table tbody tr'):
            tds = tr.find_all('td')
            msk_data.append({
                'year': y,
                'rank': tds[0].text,
                'university': tds[1].text,
                'country': tds[2].text
            })
    else:
        print(f"{msk_url} return status code {msk_resp.status_code}")

    # Бережём источник данных
    time.sleep(1)

msk_df = pd.DataFrame(msk_data)
msk_df.to_csv(f"data/msk_{min(years)}-{max(years)}.csv", index=False)
msk_df.to_excel(f"data/msk_{min(years)}-{max(years)}.xlsx", index=False)
msk_df.head(5)

,year,rank,university,country
0,2020,1,Harvard University,США
1,2020,2,Massachusetts Institute of Technology,США
2,2020,3,University of Cambridge,Великобритания
3,2020,4,University of Oxford,Великобритания
4,2020,5,University of Pennsylvania,США


Данные Московского рейтинга получены. Давайте посмотрим представительство стран по годам (ТОП 10).

In [6]:
msk_pivot = pd.pivot_table(
    msk_df,
    index='country',
    columns='year',
    values='university',
    aggfunc='count'
)
msk_pivot['sum'] = msk_pivot.sum(axis=1)
msk_pivot.sort_values(by='sum', ascending=False).head(10)

year,2020,2021,2022,sum
country,,,,
США,220.0,239.0,253.0,712.0
Китай,122.0,144.0,173.0,439.0
Россия,101.0,112.0,146.0,359.0
Великобритания,98.0,106.0,108.0,312.0
Япония,93.0,102.0,101.0,296.0
Германия,69.0,73.0,74.0,216.0
Италия,50.0,54.0,54.0,158.0
Индия,45.0,51.0,58.0,154.0
Испания,42.0,55.0,56.0,153.0
